In [18]:
from google.cloud import bigquery
import pandas

project='' #setup gcp project name/id
client = bigquery.Client(project=project)

In [20]:
datasets = list(client.list_datasets()) #get all datasets available in bq [dependency]

In [21]:
#get all table columns name and type of all tables available in bq
# dfs=[]
# for dataset in datasets:
#     dname=dataset.dataset_id
#     print("\t{}".format(dataset.dataset_id))
#     dfs.append(
#         client.query(f'SELECT table_schema, table_name, column_name, is_nullable, data_type FROM `{project}.{dname}.INFORMATION_SCHEMA.COLUMNS`').to_dataframe()
#     )

# all_table_columns=pandas.concat(dfs)
#all_table_columns[all_table_columns['column_name'] == 'have_more_than_one_status'] # verify of what table is one column name

In [22]:
#get all table and views name of all datasets in bq
dfs=[]
for dataset in datasets:
    dname=dataset.dataset_id
    print("\t{}".format(dataset.dataset_id))
    dfs.append(
        client.query(f'SELECT * FROM `{project}.{dname}.__TABLES__`').to_dataframe()
    )

all_t_and_views=pandas.concat(dfs)

	playground
	raw_data
	ready_for_use
	staging_data


In [23]:
#get all procedure names and sql of all datasets in bq
dfs=[]
for dataset in datasets:
    dname=dataset.dataset_id
    print("\t{}".format(dataset.dataset_id))
    dfs.append(
        client.query(f'SELECT * FROM `{project}.{dname}.INFORMATION_SCHEMA.ROUTINES`').to_dataframe()
    )

all=pandas.concat(dfs)
all['routine_id']=all[['routine_catalog', 'routine_schema', 'routine_name']].agg('.'.join, axis=1)

	playground
	raw_data
	ready_for_use
	staging_data


In [24]:
views=all_t_and_views[all_t_and_views['type'] == 2] #depends on #get all table and views name of all datasets in bq
views['view_id']=views[['project_id', 'dataset_id', 'table_id']].agg('.'.join, axis=1)

#get all views and its query from bq
view_ids=list(views['view_id'])
a=[]
for view_id in view_ids:
    try:
        view = client.get_table(view_id)
        a.append((view_id, view.view_query))
    except:
        print(f'ERROR: maybe the view does not exist // {view_id}')

all_views=pandas.DataFrame(columns=['view_id', 'query'], data=a)

all_views['view_name']=all_views['view_id'].apply(lambda x:x.split('.')[2])

C:\Users\jessica.mitiko_dp6\AppData\Local\Temp\ipykernel_17036\219949201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  views['view_id']=views[['project_id', 'dataset_id', 'table_id']].agg('.'.join, axis=1)


In [25]:
# all[all['routine_definition'].str.contains('exp_transactions.A2_cleaned_syn_view')][['routine_name', 'specific_schema']] #get all procedures where this substring exists
# all_views[all_views['query'].str.contains('exp_transactions.A2_cleaned_syn_view')][['view_name', 'view_id']] #get all views where this substring exists

#get all procedures and views where one or more substrings in the list exists
# tbs=['dataset.view_name', 'dataset.procedure_name']
# for tb in tbs:
#     r=all[all['routine_definition'].str.contains(tb)]['routine_name']
#     v=all_views[all_views['query'].str.contains(tb)]['view_name']

#     print(f'\n\n{tb} \nr: {r} \nv: {v}')


In [26]:
#get all table, view and procedure path id to use as source vertices for the graph
import re
all_t_and_views['table_id_without_numeric_suffix']=all_t_and_views['table_id'].apply(lambda x:re.sub(r'_[0-9]+$', '_', x)) #remove date suffix from all tables where is present

dfs=[]
#all tables and views
dfs.append(
    all_t_and_views[['project_id', 'dataset_id', 'table_id_without_numeric_suffix']].agg('.'.join, axis=1))
#all procedures
dfs.append(
    all[['routine_catalog', 'routine_schema', 'routine_name']].agg('.'.join, axis=1))

r_all=list(pandas.concat(dfs).unique()) #all table, view and procedure path id

In [27]:
#get all nodes and edges [important!] external tables will NOT appear as a source so it will not appear in the graph, this occurs because all sources (tables, views and procedures) are query from {project}, but if you need to see external relations you can query others projects and concat it on one dataframe (r_all / real_all)
edges=[]
titles=[]

tbs=list(r_all)
for tb in tbs:
    r=all[all['routine_definition'].str.contains(tb)]['routine_id'] #dependent routines
    v=all_views[all_views['query'].str.contains(tb)]['view_id'] #dependent views

    dependencies = list(v) + list(r) 
    #f=lambda x:x #do nothing
    get_name=lambda x:re.search(r'[A-Za-z0-9_-]+$', x).group(0) #get view, procedure or table name
    get_dataset=lambda x:re.search(r'^[a-zA-Z0-9-]+\.([A-Za-z_]+)', x).group(1)
    #            (source,          destination,    edge title                                  )
    edges=edges+[(get_dataset(tb), get_dataset(d), f'{get_name(d)}\n [...] FROM {get_name(tb)}') for d in dependencies if not tb == d] # [(source, dependency)]

#for dataset graph remove circular reference
edges=list(set(edges))
edges=[(a, b, c) for (a, b, c) in edges if not a == b]

In [ ]:
#without edge title
# import networkx as nx
# from pyvis.network import Network

# DG = nx.DiGraph()
# DG.add_edges_from(edges)

# net=Network(notebook=True, cdn_resources='in_line', directed =True)
# net.from_nx(DG)

# net.show('example.html')

In [30]:
#with edge title
import networkx as nx
from pyvis.network import Network

DG = nx.DiGraph()

for x, y, z in edges:
    DG.add_edge(x, y, title=z)

net=Network(notebook=True, cdn_resources='in_line', directed=True)
net.from_nx(DG)

net.show('viz/example1.html')

viz/example1.html
